In [ ]:
%%capture
! pip install kaggle
! mkdir ~/.kaggle
! cp kaggle.json ~/.kaggle/
! chmod 600 ~/.kaggle/kaggle.json
! kaggle datasets download dansbecker/hot-dog-not-hot-dog 
! unzip hot-dog-not-hot-dog.zip

In [ ]:
%%capture
!pip install -q tflite-model-maker
!pip uninstall opencv-python-headless 
!pip install opencv-python-headless==4.1.2.30
!pip install tensorflow
!pip install wandb

In [ ]:
import os
import numpy as np
import tensorflow as tf
assert tf.__version__.startswith('2')
from tflite_model_maker import model_spec
from tflite_model_maker import image_classifier
from tflite_model_maker.image_classifier import DataLoader
import tensorflow as tf
assert tf.__version__.startswith('2')
import matplotlib.pyplot as plt
import numpy as np
import os
import cv2

import seaborn as sns

# Importing all necessary libraries
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense
from keras import backend as K
 
import tensorflow as tf
from tensorflow.keras.applications import EfficientNetB0

In [ ]:
from tensorflow.keras.layers import Input, Lambda, Dense, Flatten
from tensorflow.keras.models import Model
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.applications.vgg19 import VGG19
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential\

In [ ]:
import wandb
from wandb.keras import WandbCallback

wandb.login()

In [ ]:
IMG_SIZE = 128

In [ ]:
labels = ['hot_dog', 'not_hot_dog']
img_size = IMG_SIZE
def get_data(data_dir):
    data = [] 
    for label in labels: 
        path = os.path.join(data_dir, label)
        class_num = labels.index(label)
        for img in os.listdir(path):
            try:
                img_arr = cv2.imread(os.path.join(path, img))[...,::-1] #convert BGR to RGB format
                resized_arr = cv2.resize(img_arr, (img_size, img_size)) # Reshaping images to preferred size
                data.append([resized_arr, class_num])
            except Exception as e:
                print(e)
    return np.array(data)

In [ ]:
train = '/content/train'
test = '/content/test'

In [ ]:
input_shape = (IMG_SIZE, IMG_SIZE, 3)

In [ ]:
model = Sequential()
model.add(Conv2D(64, (2, 2), input_shape=input_shape))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(32, (2, 2), input_shape=input_shape))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
 
model.add(Flatten())
model.add(Dense(16))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(1))
model.add(Activation('sigmoid'))

In [ ]:
train_datagen = ImageDataGenerator(
    rescale=1. / 255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True)
 
test_datagen = ImageDataGenerator(rescale=1. / 255)
 
train_generator = train_datagen.flow_from_directory(
    train,
    target_size=(IMG_SIZE, IMG_SIZE),
    batch_size=32,
    class_mode='binary')
 
validation_generator = test_datagen.flow_from_directory(
    test,
    target_size=(IMG_SIZE, IMG_SIZE),
    batch_size=32,
    class_mode='binary')
 


In [ ]:
wandb.init(entity='<your entity here>', project='SeeFood')

# Create a VGG16 model, and removing the last layer that is classifying 1000 images. This will be replaced with images classes we have. 
vgg = VGG16(input_shape=input_shape, weights='imagenet', include_top=False) #Training with Imagenet weights# Use this line for VGG19 network. Create a VGG19 model, and removing the last layer that is classifying 1000 images. This will be replaced with images classes we have. 
#vgg = VGG19(input_shape=IMAGE_SIZE + [3], weights='imagenet', include_top=False)# This sets the base that the layers are not trainable. If we'd want to train the layers with custom data, these two lines can be ommitted. 
for layer in vgg.layers:
  layer.trainable = False

x = Flatten()(vgg.output) #Output obtained on vgg16 is now flattened. 
prediction = Dense(2, activation='softmax')(x) # We have 5 classes, and so, the prediction is being done on len(folders) - 5 classes#Creating model object 
model = Model(inputs=vgg.input, outputs=prediction)

model.compile(loss='sparse_categorical_crossentropy', optimizer='rmsprop', metrics=['accuracy']) 

history = model.fit(train_generator, validation_data=validation_generator, epochs=50, batch_size=16, 
          callbacks=[WandbCallback(data_type="image", validation_data=validation_generator), 
                     tf.keras.callbacks.EarlyStopping(patience=10, restore_best_weights=True)])

wandb.finish()

In [ ]:
# Convert the model.
converter = tf.lite.TFLiteConverter.from_keras_model(model)
converter.post_training_quantize=True
tflite_model = converter.convert()

# Save the model.
with open('model.tflite', 'wb') as f:
  f.write(tflite_model)


In [ ]:
tflite_convert --graph_def_file=optimized_graph.pb \
  --output_file=output/optimized_graph_quantized.tflite \
  --output_format=TFLITE \
  --input_shape=1,299,299,3 \
  --input_array=Mul \
  --output_array=final_result \
  --inference_type=QUANTIZED_UINT8 \
  --std_dev_values=128 --mean_values=128 \
  --default_ranges_min=-6 --default_ranges_max=6 \
  --quantize_weights=true
